In [ ]:
from neo4j import GraphDatabase
from db_inference import simple_sql_db
from functools import lru_cache

from more_itertools import chunked
from utils import notebook_util
notebook_util.disp_notebook_full_width()
NEO4J_DOCKER_INSTANCE_NAME = "testneo4j"

In [ ]:
sql_db = simple_sql_db.SimpleSqlDb()
sql_db.get_tables()

In [ ]:
# %%time

# # @lru_cache(maxsize=1000000)
# def p100_to_p30_cached(p100):
#     row = sql_db.get_p30_cluster_for_p100(p100)
#     if row is None:
#         return row
#     else:
#         row['p30']

# cur = sql_db.conn.cursor()
# num_rows = 0
# num_cluster_misses = 0
# for r in cur.execute(f"SELECT * FROM prot2protwindow LIMIT 1000;"):
#     print(r['p1hash'], r['p2hash'])
#     p30_a = p100_to_p30_cached(r['p1hash'])
#     p30_b = p100_to_p30_cached(r['p2hash'])
    
#     if (p30_a is None) or (p30_b is None):
#         num_cluster_misses+=1
#         # p30 miss!
#         continue
#     num_rows += 1
    
# print(num_rows, num_cluster_misses)





In [ ]:
# https://towardsdatascience.com/create-a-graph-database-in-neo4j-using-python-4172d40f89c4
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response


conn = Neo4jConnection(uri=f"bolt://{NEO4J_DOCKER_INSTANCE_NAME}:7687", 
                       user="neo4j",              
                       pwd="test")

In [ ]:
# add uniqueness constraints
conn.query('CREATE CONSTRAINT ON (p:P30) ASSERT p.p30 IS UNIQUE')
conn.query('CREATE CONSTRAINT ON (p:P90) ASSERT p.p90 IS UNIQUE')
conn.query('CREATE CONSTRAINT ON (p:P100) ASSERT p.p100 IS UNIQUE')

In [ ]:
cur = sql_db.conn.cursor()
for r in cur.execute(f"SELECT * FROM prot2protwindow LIMIT 1000;"):
    print(r['p1hash'], r['p2hash'])

#     # create node 1
#     cipher_q = "CREATE (n:P100 {p100: '%s'})" % r['p1hash']
#     print(cipher_q)
#     conn.query(cipher_q)

# #     # create node 2
#     cipher_q = "CREATE (n:P100 {p100: '%s'})" % r['p2hash']
#     print(cipher_q)
#     conn.query(cipher_q)
    
    # can't store bidirectional relationship.  
    # https://stackoverflow.com/questions/43783241/why-neo4j-dont-allows-not-directed-or-bidirectional-relationships-at-creation-t/43793964
    cipher_q = "MATCH (n:P100 {p100: '%s'}), (m:P100 {p100: '%s'}) MERGE (n)-[:WINDOWED_NEIGHBOR]->(m)" % (r['p1hash'], r['p2hash'])
    print(cipher_q)
    conn.query(cipher_q)
    

In [ ]:
cur = sql_db.conn.cursor()
rows = [dict(x) for x in cur.execute(f"SELECT * FROM prot2protwindow LIMIT 10000;")]
cur.close()

query = '''
        UNWIND $rows AS row
        MATCH (n:P100 {p100: row.p1hash}), (m:P100 {p100: row.p2hash})
        MERGE (n)-[:WINDOWED_NEIGHBOR]->(m)
        RETURN count(*) as total
        '''
conn.query(query, parameters = {'rows':rows})



In [ ]:
%%time

cur = sql_db.conn.cursor()
cur.execute(f"SELECT * FROM prot2protwindow LIMIT 1000000;")

for rows in chunked(cur, 1000):x
#     print(len(rows))
    len(rows)

cur.close()